In [38]:
##Loading all the required libraries
import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB,GaussianNB

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
import time

#library for regular expretion
import re


In [39]:
Fb_Data1=pd.read_csv("FB_User_Classification.csv", delimiter="\t")

In [40]:
import pandas as pd
import io
df=Fb_Data1[['description', 'INDEX New']]


In [132]:
df.head()

,description,INDEX New
0,An Alladin's cave of beautiful designer brands...,No Seller
1,Everyone - let me take a minute to clarify som...,No Seller
2,CHANEL QUILTED BACKPACK SMALL 23X26CM MQH WITH...,Fake Seller
3,NaN,Reseller
4,Longchamp Zip Around Wallet PM / Whatsapp 012-...,Reseller


In [41]:
df['description'].replace('', np.nan, inplace=True)
df.head()

/home/irfanalidv/anaconda2/envs/py35/lib/python3.5/site-packages/pandas/core/generic.py:3554: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,description,INDEX New
0,An Alladin's cave of beautiful designer brands...,No Seller
1,Everyone - let me take a minute to clarify som...,No Seller
2,CHANEL QUILTED BACKPACK SMALL 23X26CM MQH WITH...,Fake Seller
3,NaN,Reseller
4,Longchamp Zip Around Wallet PM / Whatsapp 012-...,Reseller


In [42]:
df.dropna(subset=['description'], inplace=True)
len(df)

/home/irfanalidv/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


28882

In [43]:
df['INDEX New'].replace('NaN', np.nan, inplace=True)
df.dropna(subset=['INDEX New'], inplace=True)
len(df)
'NaN' in df['description']
'NaN' in df['INDEX New']

/home/irfanalidv/anaconda2/envs/py35/lib/python3.5/site-packages/pandas/core/generic.py:3554: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/irfanalidv/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


False

In [143]:
df.groupby('INDEX New').describe()

description
INDEX New                                                            
Fake Seller count                                                8309
            unique                                               5963
            top           CHANEL TO ORDER PLZ WHATSAPP =+919896117505
            freq                                                   34
No Seller   count                                               12885
            unique                                              11102
            top                          #love    #fashion #style    
            freq                                                   53
Reseller    count                                                7688
            unique                                               5956
            top     ลดกระหน่ำล้างสต๊อคกว่า50%TOPSHOP,ZARA,MNG,F21,...
            freq                                                   33

In [145]:
type(df['description'][0])

str

In [16]:
# function to clean data
import time
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
stem = LancasterStemmer()

stops = set(stopwords.words("english"))
def cleanData(text, lowercase = False, remove_stops = False, stemming = False):
    txt = str(text)
    txt = re.sub(r'[^A-Za-z\s]',r'',txt)
    txt = re.sub(r'\n',r' ',txt)
    
    if lowercase:
        txt = " ".join([w.lower() for w in txt.split()])
        
    if remove_stops:
        txt = " ".join([w for w in txt.split() if w not in stops])
    
    if stemming:
        stemmer = LancasterStemmer()
        txt = " ".join([stemmer.stem(w) for w in txt.split()])

    return txt


In [17]:
df['description'] = df['description'].map(lambda x: cleanData(x, lowercase=True, remove_stops=True, stemming=True))

/home/irfanalidv/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
df['description']

0        alladin cav beauty design brand includ pristin...
1        everyon let tak minut clar some us hor story s...
2                chanel quilt backpack smal xcm mqh box jt
4                   longchamp zip around wallet pm whatsap
5        pre ord longchamp neo new col pm detail inbox ...
6        onhand longchamp neo strap complet car card pa...
7        pract ab go kuy susuportah kit pra madevelopt ...
8        swap tradein chanel class reissu max flap sery...
9        chi indovin per primao il nom di quest bor dar...
10                                                  motown
11                                                  motown
12       em lin cholizz lin adizzyshop lac perf minidre...
13       callwhatsap ord delivery mad country wid fre w...
14       callwhatsap ord delivery mad country wid fre w...
15       wen es morg im bad mal wied schell geh muss ha...
16       wen es morg im bad mal wied schell geh muss ha...
17                                                 fun u

In [44]:
df.columns = ['description','CATEGORY']
df.head()

,description,CATEGORY
0,An Alladin's cave of beautiful designer brands...,No Seller
1,Everyone - let me take a minute to clarify som...,No Seller
2,CHANEL QUILTED BACKPACK SMALL 23X26CM MQH WITH...,Fake Seller
4,Longchamp Zip Around Wallet PM / Whatsapp 012-...,Reseller
5,Pre order Longchamp neo new color Pm for more ...,Reseller


In [46]:
#converting category column into numeric target NUM_CATEGORY column
df['NUM_CATEGORY']=df.CATEGORY.map({'No Seller':0,'Reseller':1,'Fake Seller':2})

/home/irfanalidv/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [56]:
#used in model 1,2
x=df['description']
y=df['NUM_CATEGORY']


x_train, x_test, y_train, y_test = train_test_split(x, y,train_size=.85)

vect =CountVectorizer(ngram_range=(2,2))
#converting features into numeric vector
X_train = vect.fit_transform(x_train)
#converting target into numeric vector
X_test = vect.transform(x_test)


In [59]:
#Training and Predicting the data

#start = time.clock()

mnb = MultinomialNB(alpha=0.2)

mnb.fit(X_train,y_train)

result= mnb.predict(X_test)

#print (time.clock()-start)

In [60]:
accuracy_score(result,y_test)

0.85275790445418875